# Results of the attribute inference attacks

## 1 - Load the results

In [1]:
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
from sklearn.metrics import accuracy_score
import sys

# Use this to load our modules.
module_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.helpers.utils import get_labels

In [2]:
nbr_targets = 1000
experiments_dir = '../../experiments/aia'
dataset_names = ['fifa19_v2']
attack_methods = ['cia_aia_model_based', 
                  'cia_aia_model_less', 
                  'copula_base', 
                  'csmia', 
                  'fredrikson', 
                  'yeom', 
                  'cai', 
                  'wcai', 
                  'marginal_prior', 
                  'cia_synth_wcai_v1', 
                  'cia_synth_wcai_v2',
                  'cia_synth_wcai_v3', 
                  'cia_synth_wcai_v4',
                 ]

Load the attribute inference attack results (including the target correlation matrices).

In [3]:
def load_aia_results(experiments_dir, attack_method, nbr_targets, dataset_name='fifa19_v2'):
    if dataset_name in [ 'fifa19', 'fifa19_v2' ]:
        nbr_data_samples = 2000
    else:
        nbr_data_samples = 1000
    results_path = f'{experiments_dir}/{dataset_name}/cols-4/smt-logreg_mmt-logreg/nt-{nbr_targets}/65682867/' + \
        f'results_{attack_method}_ntr-500_ns-5000_nds-{nbr_data_samples}_ndsbb-100_nb-3.pickle'
    with open(results_path, 'rb') as f:
        results = pickle.load(f)
    return results

attack_results = {dataset_name: 
                  {attack_method: load_aia_results(experiments_dir, attack_method, nbr_targets, dataset_name)
                  for attack_method in attack_methods} 
                 for dataset_name in dataset_names}

In [4]:
print(attack_results['fifa19_v2'].keys())
print(attack_results['fifa19_v2']['copula_base'].keys())
print(len(attack_results['fifa19_v2']['cia_aia_model_based']['accs']))

dict_keys(['cia_aia_model_based', 'cia_aia_model_less', 'copula_base', 'csmia', 'fredrikson', 'yeom', 'cai', 'wcai', 'marginal_prior', 'cia_synth_wcai_v1', 'cia_synth_wcai_v2', 'cia_synth_wcai_v3', 'cia_synth_wcai_v4'])
dict_keys(['X_cols', 'univariates', 'mse', 'X1_true', 'X1_pred', 'quantiles', 'binned_X1_true', 'binned_X1_pred', 'accs', 'target_correlation_matrices'])
1000


## 2 - Comparison between different methods

### Accuracy

In [5]:
import scipy.stats as st

for dataset_name in dataset_names:
    print(f'Dataset={dataset_name}')
    for attack_method in attack_methods:
        accs = attack_results[dataset_name][attack_method]['accs']
        mean_acc, std_acc = np.mean(accs), np.std(accs)
        a, b = st.t.interval(alpha=0.95, df=len(accs)-1, loc=np.mean(accs), scale=st.sem(accs))
        ci_error = mean_acc-a
        print(f'\tAttack method={attack_method}: mean and std {mean_acc:.1%}+-{std_acc:.1%}, 95% ci= error {ci_error:.1%}')

Dataset=fifa19_v2
	Attack method=cia_aia_model_based: mean and std 49.7%+-15.8%, 95% ci= error 1.0%
	Attack method=cia_aia_model_less: mean and std 38.6%+-13.6%, 95% ci= error 0.8%
	Attack method=copula_base: mean and std 41.8%+-14.3%, 95% ci= error 0.9%
	Attack method=csmia: mean and std 46.5%+-14.4%, 95% ci= error 0.9%
	Attack method=fredrikson: mean and std 38.4%+-12.4%, 95% ci= error 0.8%
	Attack method=yeom: mean and std 38.5%+-12.2%, 95% ci= error 0.8%
	Attack method=cai: mean and std 46.5%+-14.4%, 95% ci= error 0.9%
	Attack method=wcai: mean and std 41.7%+-13.2%, 95% ci= error 0.8%
	Attack method=marginal_prior: mean and std 35.3%+-10.4%, 95% ci= error 0.6%
	Attack method=cia_synth_wcai_v1: mean and std 46.7%+-15.6%, 95% ci= error 1.0%
	Attack method=cia_synth_wcai_v2: mean and std 49.5%+-15.6%, 95% ci= error 1.0%
	Attack method=cia_synth_wcai_v3: mean and std 49.8%+-15.4%, 95% ci= error 1.0%
	Attack method=cia_synth_wcai_v4: mean and std 50.5%+-14.2%, 95% ci= error 0.9%
